In [2]:
import pandas as pd
import numpy as np
import datetime

In [11]:
df = pd.read_csv('./screener.csv')
df['Date'] = pd.Timestamp.today().date()
df['Date'] = pd.to_datetime(df['Date'])

df['Short Float / Ratio'].replace('-', np.nan, inplace=True)
df[['Short Float', 'Short Float Ratio']] = df['Short Float / Ratio'].str.split(" / ", expand=True)
df.drop(columns=['Short Float / Ratio'], inplace=True)

# converting to float
percent_cols = ['Insider Own', 'Perf Week','Insider Trans', 'EPS this Y', 'Inst Trans', 'Perf Half Y', 'EPS next Y Percentage', 'ROA', 'Perf Year', 'EPS next 5Y', 'ROE', 'Perf YTD',
        'EPS past 5Y', 'ROI', '52W High', 'Dividend %', 'Sales past 5Y', 'Gross Margin', '52W Low', 'Sales Q/Q', 'Oper. Margin', 'Volatility W', 'Volatility M',
        'EPS Q/Q', 'Profit Margin', 'SMA20', 'SMA50', 'SMA200', 'Inst Own', 'Perf Quarter', 'Perf Month', 'Payout', 'Short Float']

df[percent_cols] = df[percent_cols].apply(lambda x: pd.to_numeric(x.str.replace('%', '').str.replace('-','')))

new_percent_col_names = [i+'_percentage' for i in percent_cols]
df.rename(columns=dict(zip(percent_cols, new_percent_col_names)), inplace=True)

def _convert_to_number(value):
    if value.endswith('K'):
        return float(value.strip('K')) * 1000
    elif value.endswith('M'):
        return float(value.strip('M')) * 1000_000
    elif value.endswith('B'):
        return float(value.strip('B')) * 1000_000_000
    elif value.endswith('T'):
        return float(value.strip('T')) * 1000_000_000_0000
    elif value.endswith('-'):
        return np.nan
    else:
        return float(value)
    
def _format_earnings(df):
    # Earnings to Earnings_Date and Earnings_Timing
    df['Earnings'].replace('-', np.nan, inplace=True)
    df[['Earnings_Date', 'Earnings_Timing']] = df['Earnings'].str.extract(r"([a-zA-Z]+\s\d+)\s([a-zA-Z]*)")
    current_year = str(pd.to_datetime('today').year)
    df['Earnings_Date'] = df['Earnings_Date']+ " " +current_year 
    df['Earnings_Date'] = pd.to_datetime(df['Earnings_Date'], format='%b %d %Y', errors='coerce')

    if (datetime.datetime.now().month in [1,2,3]):
        df['Earnings_Date'] = df['Earnings_Date'].apply(lambda x: x.replace(year=datetime.datetime.now().year-1) if (x - datetime.datetime.now()).days > 120 else x.replace(year=datetime.datetime.now().year))
    if (datetime.datetime.now().month in [10,11,12]):
        df['Earnings_Date'] = df['Earnings_Date'].apply(lambda x: x.replace(year=datetime.datetime.now().year+1) if (datetime.datetime.now() - x).days > 200 else x.replace(year=datetime.datetime.now().year))
    df.drop(columns=['Earnings'], inplace=True)
    return df

# converting to float
float_cols = ['Shs Outstand','Employees', 'Shs Float','Income', 'Sales', 'Short Interest' ,'Avg Volume',
                'EPS (ttm)', 'Forward P/E', 'EPS next Y', 'PEG', 'EPS next Q', 'P/S',
                'Book/sh', 'P/B', 'Target Price', 'Cash/sh', 'P/C', '52W Range From',
                'Dividend', 'P/FCF', 'Beta', 'Quick Ratio', 'Current Ratio', 'RSI (14)',
                'Debt/Eq', 'LT Debt/Eq', 'Recom', 'Short Float Ratio']
for col in float_cols:
    df[col] = df[col].apply(_convert_to_number)


int_cols = ['Shs Outstand', 'Market Cap', 'Volume', 'Employees', 'Avg Volume'] 
df[int_cols] = df[int_cols].fillna(0)
df[int_cols] = df[int_cols].astype(int)

bool_cols = ['Optionable', 'Shortable']
bool_dict = {'Yes': True, 'No': False}
df[bool_cols] = df[bool_cols].applymap(lambda x: bool_dict[x])

# Earnings to Earnings_Date and Earnings_Timing
df = _format_earnings(df)

# converting to category
cat_cols = ['Sector', 'Industry', 'Country', 'Signals', 'Index', 'Earnings_Timing']
df[cat_cols] = df[cat_cols].astype('category')

df.columns = df.columns.str.replace(' ', '_').str.replace('/', '_')
df.rename(columns={'Dividend_%_percentage': 'Dividend_percentage'}, inplace=True)


In [19]:
df.columns

Index(['Ticker', 'Company', 'Sector', 'Industry', 'Country', 'Market_Cap',
       'P_E', 'Price', 'Change', 'Volume', 'Signals', 'NoOf_Signals', 'Index',
       'EPS_(ttm)', 'Insider_Own_percentage', 'Shs_Outstand',
       'Perf_Week_percentage', 'Forward_P_E', 'EPS_next_Y',
       'Insider_Trans_percentage', 'Shs_Float', 'Perf_Month_percentage',
       'Income', 'PEG', 'EPS_next_Q', 'Inst_Own_percentage',
       'Perf_Quarter_percentage', 'Sales', 'P_S', 'EPS_this_Y_percentage',
       'Inst_Trans_percentage', 'Short_Interest', 'Perf_Half_Y_percentage',
       'Book_sh', 'P_B', 'EPS_next_Y_Percentage_percentage', 'ROA_percentage',
       'Target_Price', 'Perf_Year_percentage', 'Cash_sh', 'P_C',
       'EPS_next_5Y_percentage', 'ROE_percentage', '52W_Range_From',
       '52W_Range_To', 'Perf_YTD_percentage', 'Dividend', 'P_FCF',
       'EPS_past_5Y_percentage', 'ROI_percentage', '52W_High_percentage',
       'Beta', 'Dividend_percentage', 'Quick_Ratio',
       'Sales_past_5Y_percentage

In [37]:
from finvizfinance.screener.overview import Overview

screener = Overview()
screener_signals = screener.get_signal()
screener_filters = screener.get_filters()

dct = {}
for i in screener_filters:
    dct[i] = screener.get_filter_options(i)

dct

{'Exchange': ['Any', 'AMEX', 'NASDAQ', 'NYSE'],
 'Index': ['Any', 'S&P 500', 'DJIA'],
 'Sector': ['Any',
  'Basic Materials',
  'Communication Services',
  'Consumer Cyclical',
  'Consumer Defensive',
  'Energy',
  'Financial',
  'Healthcare',
  'Industrials',
  'Real Estate',
  'Technology',
  'Utilities'],
 'Industry': ['Any',
  'Stocks only (ex-Funds)',
  'Exchange Traded Fund',
  'Advertising Agencies',
  'Aerospace & Defense',
  'Agricultural Inputs',
  'Airlines',
  'Airports & Air Services',
  'Aluminum',
  'Apparel Manufacturing',
  'Apparel Retail',
  'Asset Management',
  'Auto Manufacturers',
  'Auto Parts',
  'Auto & Truck Dealerships',
  'Banks - Diversified',
  'Banks - Regional',
  'Beverages - Brewers',
  'Beverages - Non-Alcoholic',
  'Beverages - Wineries & Distilleries',
  'Biotechnology',
  'Broadcasting',
  'Building Materials',
  'Building Products & Equipment',
  'Business Equipment & Supplies',
  'Capital Markets',
  'Chemicals',
  'Closed-End Fund - Debt',
  'C

In [38]:
dct.keys()


dict_keys(['Exchange', 'Index', 'Sector', 'Industry', 'Country', 'Market Cap.', 'P/E', 'Forward P/E', 'PEG', 'P/S', 'P/B', 'Price/Cash', 'Price/Free Cash Flow', 'EPS growththis year', 'EPS growthnext year', 'EPS growthpast 5 years', 'EPS growthnext 5 years', 'Sales growthpast 5 years', 'EPS growthqtr over qtr', 'Sales growthqtr over qtr', 'Dividend Yield', 'Return on Assets', 'Return on Equity', 'Return on Investment', 'Current Ratio', 'Quick Ratio', 'LT Debt/Equity', 'Debt/Equity', 'Gross Margin', 'Operating Margin', 'Net Profit Margin', 'Payout Ratio', 'InsiderOwnership', 'InsiderTransactions', 'InstitutionalOwnership', 'InstitutionalTransactions', 'Float Short', 'Analyst Recom.', 'Option/Short', 'Earnings Date', 'Performance', 'Performance 2', 'Volatility', 'RSI (14)', 'Gap', '20-Day Simple Moving Average', '50-Day Simple Moving Average', '200-Day Simple Moving Average', 'Change', 'Change from Open', '20-Day High/Low', '50-Day High/Low', '52-Week High/Low', 'Pattern', 'Candlestick',

In [52]:
from finvizfinance.insider import Insider
from finvizfinance.calendar import Calendar
from finvizfinance.news import News
from finvizfinance.earnings import Earnings

def get_callendar():
    calendar = Calendar().calendar()
    return calendar

def get_news():
    news = News().get_news()
    return news

def get_insider():
    insider_list = ['latest', 'latest buys', 'latest sales', 
                    'top week', 'top week buys', 'top week sales', 
                    'top owner trade', 'top owner buys', 'top owner sales']
    for i in insider_list:
        insider = Insider(i).get_insider()
        insider['signal'] = i
    return insider

def get_earnings(base_path):
    earnings_list = ['This Week', 'Next Week', 'Previous Week','This Month']
    for i in earnings_list:
        path = "{base_path}/earnings_{i}.csv"
        earnings = Earnings(i)


        

In [57]:
Earnings('Previous Week').output_csv('./')

Print to CSV...


In [119]:
import glob
import pandas as pd

screener = glob.glob('../data/screeners/*.csv')[0:2]
description = glob.glob('../data/screeners/*.csv')[2]
description = pd.read_csv(description)
all_data = pd.DataFrame()
for i in screener:
    df = pd.read_csv(i)
    all_data = pd.concat([all_data, df], axis=0)

all_data = all_data[['Ticker', 'Company','Sector','Industry','Country',  'Description']]
all_data.drop_duplicates(inplace=True)
info = description.merge(all_data, on='Ticker', how='left')

# sublset the dataframe with Na values
mask = info['Sector'].isna()
tickers = info[mask]['Ticker'].to_list()
info = info[~mask]
info.dropna(inplace=True)
info.drop(columns=['Description_y'], inplace=True)
info.rename(columns={'Description_x': 'Description'}, inplace=True)
info.isna().sum()


Ticker         0
Description    0
Company        0
Sector         0
Industry       0
Country        0
dtype: int64

In [122]:
import time
from finvizfinance.quote import finvizfinance
description = pd.Series(name='Description', dtype='str')
for ticker in tickers:
    try:
        description[ticker] = finvizfinance(ticker).ticker_description()
        time.sleep(0.1)
    except: 
        print('error')


In [123]:
import time
from finvizfinance.quote import finvizfinance
dct = {}
for ticker in tickers:
    try:
        fundamant = finvizfinance(ticker).ticker_fundament().values()
        dct[ticker] = fundamant
        fundment = pd.DataFrame(fundamant)
        time.sleep(0.2)
    except: 
        print('error')
    
df = pd.DataFrame(dct).transpose(copy=True)
df = df.iloc[:,0:4]
df.reset_index(inplace=True, drop=False)

,index,0,1,2,3
0,ZTEK,Zentek Ltd.,Healthcare,Medical Instruments & Supplies,Canada
1,REKR,"Rekor Systems, Inc.",Industrials,Security & Protection Services,USA
2,ORGS,Orgenesis Inc.,Healthcare,Biotechnology,USA
3,MOB,Mobilicom Limited,Technology,Communication Equipment,Australia
4,NMG,Nouveau Monde Graphite Inc.,Basic Materials,Other Industrial Metals & Mining,Canada


In [124]:
df.rename(columns={'index': 'Ticker', 0: 'Company', 1: 'Sector', 2: 'Industry', 3: 'Country'}, inplace=True)


,Ticker,Company,Sector,Industry,Country
0,ZTEK,Zentek Ltd.,Healthcare,Medical Instruments & Supplies,Canada
1,REKR,"Rekor Systems, Inc.",Industrials,Security & Protection Services,USA
2,ORGS,Orgenesis Inc.,Healthcare,Biotechnology,USA
3,MOB,Mobilicom Limited,Technology,Communication Equipment,Australia
4,NMG,Nouveau Monde Graphite Inc.,Basic Materials,Other Industrial Metals & Mining,Canada


In [127]:
df['Description'] = df['Ticker'].map(description)
full_info = pd.concat([info, df], axis=0)
full_info.isna().sum()

Ticker         1
Description    0
Company        0
Sector         0
Industry       0
Country        0
dtype: int64

In [145]:
full_info = full_info[['Ticker', 'Company','Sector','Industry','Country',  'Description']]

In [146]:
full_info.to_csv('../data/screeners/ticker_info.csv', index=False)

In [28]:
from finvizfinance.news import News
import pandas as pd
news = News().get_news()
news['news'].to_csv('./tem1.csv', index=False)
news['news'].head(25)


,Date,Title,Source,Link
0,07:50PM,After Telling Millions of Taxpayers to Hold Of...,www.wsj.com,https://www.wsj.com/articles/after-telling-mil...
1,07:33PM,"As investors cheer 'disinflation,' Jamie Dimon...",finance.yahoo.com,https://finance.yahoo.com/news/jamie-dimon-say...
2,07:18PM,Wall St Week Ahead Last year's laggards lead U...,www.reuters.com,https://www.reuters.com/markets/wall-st-week-a...
3,07:11PM,Is the iPhone's 'Made in India' era about to b...,www.cnn.com,https://www.cnn.com/2023/02/10/tech/india-appl...
4,07:05PM,ChatGPT Clones Are Preparing to Take Over China,www.wsj.com,https://www.wsj.com/articles/chatgpt-clones-ar...
5,06:54PM,Why Kim Jong Un wants you to meet his daughter,www.cnn.com,https://www.cnn.com/2023/02/10/asia/kim-jong-u...
6,06:45PM,American Airlines Pilots Refuse Recorded Inter...,www.nytimes.com,https://www.nytimes.com/2023/02/10/business/nt...
7,06:30PM,S&P 500 Turns In Worst Week So Far This Year,www.wsj.com,https://www.wsj.com/articles/global-stocks-mar...
8,06:01PM,Robinhood wins dismissal of shareholder lawsui...,foxbusiness.com,https://foxbusiness.com/markets/robinhood-wins...
9,05:53PM,: Bed Bath & Beyond to close its stores in Canada,www.marketwatch.com,http://www.marketwatch.com/news/story/bed-bath...


In [27]:
news['blogs'].head(25)

,Date,Title,Source,Link
0,08:40PM,US State Department Funding Secret 'Disinforma...,www.zerohedge.com,https://www.zerohedge.com/political/us-state-d...
1,08:20PM,GoFundMe Takes Down Campaigns For Arizona Ranc...,www.zerohedge.com,https://www.zerohedge.com/political/gofundme-t...
2,08:00PM,"Republican Senator ""Can’t Rule Out"" Revelation...",www.zerohedge.com,https://www.zerohedge.com/geopolitical/republi...
3,07:40PM,Concentrate Where The Murders Are Concentrated,www.zerohedge.com,https://www.zerohedge.com/political/concentrat...
4,07:20PM,First LNG Tanker Arrives At Freeport's Texas T...,www.zerohedge.com,https://www.zerohedge.com/commodities/first-ln...
5,07:00PM,"Without Subsidies, How Many People Will Buy An...",www.zerohedge.com,https://www.zerohedge.com/personal-finance/wit...
6,06:40PM,So Much For Billionaires: Joe Biden's IRS Is N...,www.zerohedge.com,https://www.zerohedge.com/markets/so-much-bill...
7,06:20PM,9 Things You Need To Know About Paxlovid,www.zerohedge.com,https://www.zerohedge.com/medical/9-things-you...
8,05:40PM,Sanctions Made India Indispensable To The Glob...,www.zerohedge.com,https://www.zerohedge.com/geopolitical/sanctio...
9,05:39PM,Annaly Capital: Get In The Better Tranche,seekingalpha.com,https://seekingalpha.com/article/4577426-annal...


In [18]:
from finvizfinance.insider import Insider
insider = Insider().get_insider()
insider['Date'].value_counts()

Feb 08    92
Feb 09    84
Feb 10    20
Feb 02     2
Apr 18     1
Feb 07     1
Name: Date, dtype: int64

In [21]:
from finvizfinance.insider import Insider
insider = Insider('latest buys').get_insider()
insider

,Ticker,SEC Form 4 Link,Owner,Relationship,Date,Transaction,Cost,#Shares,Value ($),#Shares Total,SEC Form 4
0,CRWD,http://www.sec.gov/Archives/edgar/data/1535527...,Henry Shawn,CHIEF SECURITY OFFICER,Apr 18,Buy,229.11,2.0,458.0,170125.0,Feb 10 08:00 PM
1,RBOT,http://www.sec.gov/Archives/edgar/data/1812173...,Liang Philip,Director,Feb 08,Buy,3.05,79009.0,240740.0,162000.0,Feb 10 07:29 PM
2,PDLB,http://www.sec.gov/Archives/edgar/data/1874071...,Gonzalez Luis Gerardo Jr.,Chief Operating Officer,Feb 09,Buy,9.25,1081.0,10002.0,1306.0,Feb 10 05:23 PM
3,AKAM,http://www.sec.gov/Archives/edgar/data/1086222...,LEIGHTON F THOMSON,Chief Executive Officer,Feb 10,Buy,86.26,290.0,25015.0,12106.0,Feb 10 05:16 PM
4,AKAM,http://www.sec.gov/Archives/edgar/data/1086222...,LEIGHTON F THOMSON,Chief Executive Officer,Feb 09,Buy,87.03,288.0,25065.0,11816.0,Feb 10 05:16 PM
...,...,...,...,...,...,...,...,...,...,...,...
195,FUNC,http://www.sec.gov/Archives/edgar/data/763907/...,Rodeheaver Carissa Lynn,"Chairman, President & CEO",Feb 02,Buy,19.50,185.0,3608.0,36607.0,Feb 06 11:23 AM
196,RCG,http://www.sec.gov/Archives/edgar/data/919567/...,STAHL MURRAY,President and CEO,Feb 03,Buy,1.83,408.0,747.0,207733.0,Feb 06 11:18 AM
197,RCG,http://www.sec.gov/Archives/edgar/data/919567/...,STAHL MURRAY,President and CEO,Feb 03,Buy,1.83,1.0,2.0,3693.0,Feb 06 11:18 AM
198,TPL,http://www.sec.gov/Archives/edgar/data/1056823...,STAHL MURRAY,Director,Feb 03,Buy,2012.50,18.0,36225.0,197502.0,Feb 06 10:59 AM


In [30]:
from finvizfinance import quote
tick = quote.finvizfinance(ticker='googl')
tick.ticker_news()

,Date,Title,Link
0,2023-02-10 19:05:00,Google cautions against 'hallucinating' chatbo...,https://finance.yahoo.com/news/google-cautions...
1,2023-02-10 19:00:00,Google cautions against 'hallucinating' chatbo...,https://finance.yahoo.com/news/google-cautions...
2,2023-02-10 18:44:00,Weekly Roundup,https://finance.yahoo.com/m/098e6cbc-7d70-3543...
3,2023-02-10 18:06:00,ChatGPT Sparked an AI Craze. Investors Need a ...,https://finance.yahoo.com/m/6f5e3b38-d26b-37fb...
4,2023-02-10 17:49:00,Bill Gates says ChatGPT will change our world ...,https://finance.yahoo.com/news/bill-gates-says...
...,...,...,...
95,2023-02-08 16:09:00,Microsofts Bing is the first threat to Googles...,https://finance.yahoo.com/news/microsofts-bing...
96,2023-02-08 16:05:00,Microsoft Stock Upgraded On AI News Amid Fears...,https://finance.yahoo.com/m/8dcaa773-3961-3a01...
97,2023-02-08 15:27:00,Google shares fall sharply after AI chatbot de...,https://finance.yahoo.com/m/22cdd1dc-d571-3c8e...
98,2023-02-08 15:27:00,Googles stock dips on demonstration of its AI ...,https://finance.yahoo.com/m/3e370fe3-8372-3643...
